In [70]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from npj_utils import *
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb

In [72]:
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [73]:
import pickle

base_name = "pheno" # ihm, los, pheno

if "pheno" in base_name:
    base_name += "-all"
else:
    base_name += "-48"

# base_name += "-cxr-notes-ecg-missingInd"
base_name += "-cxr-notes-missingInd"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    train_stays = pickle.load(f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    val_stays = pickle.load(f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    test_stays = pickle.load(f)

In [74]:
include_notes = True
include_cxr = True
include_ecg = False

In [75]:
X_train = calc_ts_embeddings(train_stays)

if include_notes:
    txt_df = calc_avg_text_embedding(train_stays)
    X_train = pd.concat([X_train, txt_df], axis=1)

if include_cxr:
    cxr_df = calc_avg_cxr_embedding(train_stays) 
    X_train = pd.concat([X_train, cxr_df], axis=1)

if include_ecg:
    ecg_df = calc_avg_ecg_embedding(train_stays)
    X_train = pd.concat([X_train, ecg_df], axis=1)

y_train = extract_labels(train_stays)

col_names = X_train.columns

Calculating Time Series Embeddings:   0%|          | 0/10178 [00:00<?, ?it/s]

Calculating CXR Embeddings: 100%|██████████| 10178/10178 [15:07<00:00, 11.21it/s]


In [76]:
X_test = calc_ts_embeddings(test_stays)

if include_notes:
    txt_df = calc_avg_text_embedding(test_stays)
    X_test = pd.concat([X_test, txt_df], axis=1)

if include_cxr:
    cxr_df = calc_avg_cxr_embedding(test_stays)
    X_test = pd.concat([X_test, cxr_df], axis=1)

if include_ecg:
    ecg_df = calc_avg_ecg_embedding(test_stays)
    X_test = pd.concat([X_test, ecg_df], axis=1)

y_test = extract_labels(test_stays)

Calculating Time Series Embeddings:   0%|          | 0/2181 [00:00<?, ?it/s]

Calculating CXR Embeddings: 100%|██████████| 2181/2181 [00:18<00:00, 116.11it/s]


In [77]:
X_val = calc_ts_embeddings(val_stays)

if include_notes:
    txt_df = calc_avg_text_embedding(val_stays)
    X_val = pd.concat([X_val, txt_df], axis=1)

if include_cxr:
    cxr_df = calc_avg_cxr_embedding(val_stays)
    X_val = pd.concat([X_val, cxr_df], axis=1)

if include_ecg:
    ecg_df = calc_avg_ecg_embedding(val_stays)
    X_val = pd.concat([X_val, ecg_df], axis=1)

y_val = extract_labels(val_stays)

Calculating CXR Embeddings: 100%|██████████| 2181/2181 [00:20<00:00, 107.54it/s]


In [78]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [79]:
from prediction_util import run_xgb, run_xgb_multilabel

seed = 1
if "pheno" in base_name:
    y_pred_test, y_pred_prob_test, y_pred_train, y_pred_prob_train, gs = run_xgb_multilabel(X_train, y_train, X_test, gpu=0, seed=seed, n_jobs=16)
else:
    y_pred_test, y_pred_prob_test, y_pred_train, y_pred_prob_train, gs = run_xgb(X_train, y_train, X_test, gpu=0, seed=seed, n_jobs=16)

# Pheno (no ECG)

In [81]:
print(y_pred_test[0].shape)
print(y_pred_prob_test[0].shape)
print(y_pred_train[0].shape)
print(y_pred_prob_train[0].shape)
print(gs.best_params_)

(25,)
(2181, 2)
(25,)
(10178, 2)
{'estimator__learning_rate': 0.3, 'estimator__max_depth': 5, 'estimator__n_estimators': 200}


In [89]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Evaluate for each label
# Assuming y_test is of shape (n_samples, n_labels)
for i in range(y_test.shape[1]):  # Iterate over each label
    print(f"Evaluating label {i+1}")
    
    y_test_label = y_test[:, i]  # Extract the true labels for label i
    y_pred_label = y_pred_test[:, i]  # Extract the predicted labels for label i
    y_pred_prob_label = y_pred_prob_test[i][:, 1]  # Extract the probability for the positive class (1) for label i
    
    # Evaluate metrics
    accuracy = accuracy_score(y_test_label, y_pred_label)
    auc = roc_auc_score(y_test_label, y_pred_prob_label)  # Check shape of y_pred_prob_label
    f1 = f1_score(y_test_label, y_pred_label)
    precision = precision_score(y_test_label, y_pred_label)
    recall = recall_score(y_test_label, y_pred_label)

    print(f"Accuracy for label {i+1}: {accuracy}")
    print(f"AUC for label {i+1}: {auc}")
    print(f"F1 Score for label {i+1}: {f1}")
    print(f"Precision for label {i+1}: {precision}")
    print(f"Recall for label {i+1}: {recall}")

# Optionally, aggregate metrics across all labels (macro or micro averaging)
macro_f1 = f1_score(y_test, y_pred_test, average='macro')
micro_f1 = f1_score(y_test, y_pred_test, average='micro')
print(f"Macro F1 Score: {macro_f1}")
print(f"Micro F1 Score: {micro_f1}")

Evaluating label 1
Accuracy for label 1: 0.8055937643282898
AUC for label 1: 0.8316972298088178
F1 Score for label 1: 0.5664621676891616
Precision for label 1: 0.647196261682243
Recall for label 1: 0.5036363636363637
Evaluating label 2
Accuracy for label 2: 0.7725813846859239
AUC for label 2: 0.6207606993238511
F1 Score for label 2: 0.11743772241992882
Precision for label 2: 0.308411214953271
Recall for label 2: 0.07252747252747253
Evaluating label 3
Accuracy for label 3: 0.8702430077945896
AUC for label 3: 0.7248408583500933
F1 Score for label 3: 0.22465753424657536
Precision for label 3: 0.5189873417721519
Recall for label 3: 0.14335664335664336
Evaluating label 4
Accuracy for label 4: 0.619440623567171
AUC for label 4: 0.6549821357681619
F1 Score for label 4: 0.5053635280095352
Precision for label 4: 0.5564304461942258
Recall for label 4: 0.462882096069869
Evaluating label 5
Accuracy for label 5: 0.6790463090325539
AUC for label 5: 0.7518983509783971
F1 Score for label 5: 0.67411545

In [92]:
# Calculate AUC (micro)
avg_auc_micro = roc_auc_score(y_test, y_pred_test, average='micro')

# Calculate AUC (macro)
avg_auc_macro = roc_auc_score(y_test, y_pred_test, average='macro')

# Calculate AUC (weighted)
avg_auc_weighted = roc_auc_score(y_test, y_pred_test, average='weighted')

print(f'AUC (micro): {avg_auc_micro}')
print(f'AUC (macro): {avg_auc_macro}')
print(f'AUC (weighted): {avg_auc_weighted}')

AUC (micro): 0.6809527390950785
AUC (macro): 0.6277059636191837
AUC (weighted): 0.6520082985302481


In [93]:
est = MultiOutputClassifier(xgb.XGBClassifier(verbosity=2, seed=42,
                                                  tree_method='gpu_hist', gpu_id=1,
                                                  eval_metric='logloss', n_jobs=32,learning_rate= 0.3, max_depth= 5, n_estimators= 200))
est.fit(X_train, y_train)
y_pred_prob_test = est.predict_proba(X_test)
y_pred_test = est.predict(X_test)

In [94]:
# Optionally, aggregate metrics across all labels (macro or micro averaging)
macro_f1 = f1_score(y_test, y_pred_test, average='macro')
micro_f1 = f1_score(y_test, y_pred_test, average='micro')
print(f"Macro F1 Score: {macro_f1}")
print(f"Micro F1 Score: {micro_f1}")

Macro F1 Score: 0.4059348382973209
Micro F1 Score: 0.5230131062177309


In [95]:
# Calculate AUC (micro)
avg_auc_micro = roc_auc_score(y_test, y_pred_test, average='micro')

# Calculate AUC (macro)
avg_auc_macro = roc_auc_score(y_test, y_pred_test, average='macro')

# Calculate AUC (weighted)
avg_auc_weighted = roc_auc_score(y_test, y_pred_test, average='weighted')

print(f'AUC (micro): {avg_auc_micro}')
print(f'AUC (macro): {avg_auc_macro}')
print(f'AUC (weighted): {avg_auc_weighted}')

AUC (micro): 0.6809527390950785
AUC (macro): 0.6277059636191837
AUC (weighted): 0.6520082985302481


# LOS (no ECG)

In [45]:
print(y_pred_test.shape)
print(y_pred_prob_test.shape)
print(y_pred_train.shape)
print(y_pred_prob_train.shape)
print(gs.best_params_)

(5270,)
(5270, 2)
(24591,)
(24591, 2)
{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}


In [46]:
est = xgb.XGBClassifier(verbosity=2, seed=seed,eval_metric='logloss', n_jobs=32,learning_rate= 0.05, max_depth= 5, n_estimators= 300, device="cuda")
est.fit(X_train, y_train)

y_pred_test = est.predict(X_test)
y_pred_prob_test = est.predict_proba(X_test)

# Evaluate
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test[:,1])

Proportion of outcome: 0.47324478178368123
F1 Score: 0.7402229359531457
Accuracy: 0.7390891840607211
Balanced Accuracy: 0.741445752226092
AUC: 0.8170199256313135
AUPRC: 0.7746974031364304
Confusion Matrix: [[1936  840]
 [ 535 1959]]


In [47]:
# Make predictions on the training set
y_pred_prob_train = est.predict_proba(X_train)
y_pred_train = est.predict(X_train)

# F1 Score for the training set
f1_train = f1_score(y_train, y_pred_train)
print(f"F1 Score for Training Set is: {f1_train}")

# Accuracy for the training set
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy for Training Set is: {accuracy_train}")

# Balanced Accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_pred_train)
print(f"Balanced Accuracy for Training Set is: {balanced_accuracy_train}")

# AUC for the training set
# Ensure that y_pred_prob_train[:, 1] exists if it's a binary classification
if len(np.unique(y_train)) == 2:
    auc_train = roc_auc_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUC for Training Set is: {auc_train}")
else:
    print("AUC is typically used for binary classification. Adjust the code for multi-class if needed.")

# AUPRC for the training set
if len(np.unique(y_train)) == 2:
    auprc_train = average_precision_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUPRC for Training Set is: {auprc_train}")
else:
    print("AUPRC is typically used for binary classification. Adjust the code for multi-class if needed.")

# Confusion Matrix for the training set
cm_train = confusion_matrix(y_train, y_pred_train)
print(f"Confusion Matrix for Training Set is: \n{cm_train}")


F1 Score for Training Set is: 0.8371074939308314
Accuracy for Training Set is: 0.8335569923955919
Balanced Accuracy for Training Set is: 0.8356921438594688
AUC for Training Set is: 0.9163461677710176
AUPRC for Training Set is: 0.9007149201542283
Confusion Matrix for Training Set is: 
[[ 9981  2796]
 [ 1297 10517]]


In [48]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score, confusion_matrix

# Assumed correction: Ensure y_pred_prob_test is fetched correctly and has two columns
# y_pred_prob_test = model.predict_proba(X_test)  # Correctly fetching test probabilities

# F1 Score
f1 = f1_score(y_test, y_pred_test)
print(f"F1 Score for Testing Set is: {f1}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for Testing Set is: {accuracy}")

# Balanced Accuracy
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print(f"Balanced Accuracy for Testing Set is: {balanced_accuracy}")

# AUC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:  # Ensuring binary classification with two probability outputs
    auc = roc_auc_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUC for Testing Set is: {auc}")
else:
    print("Error: Expected binary class probabilities for AUC calculation.")

# AUPRC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:
    auprc = average_precision_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUPRC for Testing Set is: {auprc}")
else:
    print("Error: Expected binary class probabilities for AUPRC calculation.")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print(f"Confusion Matrix for Testing Set is: \n{cm}")

F1 Score for Testing Set is: 0.7402229359531457
Accuracy for Testing Set is: 0.7390891840607211
Balanced Accuracy for Testing Set is: 0.741445752226092
AUC for Testing Set is: 0.8170199256313135
AUPRC for Testing Set is: 0.7746974031364304
Confusion Matrix for Testing Set is: 
[[1936  840]
 [ 535 1959]]


# IHM (no ECG)

In [56]:
print(y_pred_test.shape)
print(y_pred_prob_test.shape)
print(y_pred_train.shape)
print(y_pred_prob_train.shape)
print(gs.best_params_)


(5270,)
(5270,)
(24591,)
(24591,)
{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200}


In [66]:
est = xgb.XGBClassifier(verbosity=2, seed=seed,eval_metric='logloss', n_jobs=32,learning_rate= 0.05, max_depth= 5, n_estimators= 200, device="cuda")
est.fit(X_train, y_train)

y_pred_test = est.predict(X_test)
y_pred_prob_test = est.predict_proba(X_test)

# Evaluate
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test[:,1])

Proportion of outcome: 0.14193548387096774
F1 Score: 0.3940242763772176
Accuracy: 0.8768500948766603
Balanced Accuracy: 0.6286588798198706
AUC: 0.8554489696244386
AUPRC: 0.5165519978746538
Confusion Matrix: [[4410  112]
 [ 537  211]]


In [67]:
# Make predictions on the training set
y_pred_prob_train = est.predict_proba(X_train)
y_pred_train = est.predict(X_train)

# F1 Score for the training set
f1_train = f1_score(y_train, y_pred_train)
print(f"F1 Score for Training Set is: {f1_train}")

# Accuracy for the training set
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy for Training Set is: {accuracy_train}")

# Balanced Accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_pred_train)
print(f"Balanced Accuracy for Training Set is: {balanced_accuracy_train}")

# AUC for the training set
# Ensure that y_pred_prob_train[:, 1] exists if it's a binary classification
if len(np.unique(y_train)) == 2:
    auc_train = roc_auc_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUC for Training Set is: {auc_train}")
else:
    print("AUC is typically used for binary classification. Adjust the code for multi-class if needed.")

# AUPRC for the training set
if len(np.unique(y_train)) == 2:
    auprc_train = average_precision_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUPRC for Training Set is: {auprc_train}")
else:
    print("AUPRC is typically used for binary classification. Adjust the code for multi-class if needed.")

# Confusion Matrix for the training set
cm_train = confusion_matrix(y_train, y_pred_train)

F1 Score for Training Set is: 0.6337464251668257
Accuracy for Training Set is: 0.9218819893456955
Balanced Accuracy for Training Set is: 0.7369427993874346
AUC for Training Set is: 0.9487933938991525
AUPRC for Training Set is: 0.8271620505190398


In [69]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score, confusion_matrix

# Assumed correction: Ensure y_pred_prob_test is fetched correctly and has two columns
# y_pred_prob_test = model.predict_proba(X_test)  # Correctly fetching test probabilities

# F1 Score
f1 = f1_score(y_test, y_pred_test)
print(f"F1 Score for Testing Set is: {f1}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for Testing Set is: {accuracy}")

# Balanced Accuracy
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print(f"Balanced Accuracy for Testing Set is: {balanced_accuracy}")

# AUC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:  # Ensuring binary classification with two probability outputs
    auc = roc_auc_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUC for Testing Set is: {auc}")
else:
    print("Error: Expected binary class probabilities for AUC calculation.")

# AUPRC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:
    auprc = average_precision_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUPRC for Testing Set is: {auprc}")
else:
    print("Error: Expected binary class probabilities for AUPRC calculation.")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print(f"Confusion Matrix for Testing Set is: \n{cm}")

F1 Score for Testing Set is: 0.3940242763772176
Accuracy for Testing Set is: 0.8768500948766603
Balanced Accuracy for Testing Set is: 0.6286588798198706
AUC for Testing Set is: 0.8554489696244386
AUPRC for Testing Set is: 0.5165519978746538
Confusion Matrix for Testing Set is: 
[[4410  112]
 [ 537  211]]


In [28]:
est = xgb.XGBClassifier(verbosity=2, seed=seed,eval_metric='logloss', n_jobs=32,learning_rate= 0.05, max_depth= 5, n_estimators= 200, device="cuda")
est.fit(X_train, y_train)

y_pred_test = est.predict(X_test)
y_pred_prob_test = est.predict_proba(X_test)

# Evaluate
_ = evaluate_model(y_test, y_pred_test, y_pred_prob_test[:,1])

Proportion of outcome: 0.14193548387096774
F1 Score: 0.3940242763772176
Accuracy: 0.8768500948766603
Balanced Accuracy: 0.6286588798198706
AUC: 0.8554489696244386
AUPRC: 0.5165519978746538
Confusion Matrix: [[4410  112]
 [ 537  211]]


In [29]:
# Make predictions on the training set
y_pred_prob_train = est.predict_proba(X_train)
y_pred_train = est.predict(X_train)

# F1 Score for the training set
f1_train = f1_score(y_train, y_pred_train)
print(f"F1 Score for Training Set is: {f1_train}")

# Accuracy for the training set
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy for Training Set is: {accuracy_train}")

# Balanced Accuracy for the training set
balanced_accuracy_train = balanced_accuracy_score(y_train, y_pred_train)
print(f"Balanced Accuracy for Training Set is: {balanced_accuracy_train}")

# AUC for the training set
# Ensure that y_pred_prob_train[:, 1] exists if it's a binary classification
if len(np.unique(y_train)) == 2:
    auc_train = roc_auc_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUC for Training Set is: {auc_train}")
else:
    print("AUC is typically used for binary classification. Adjust the code for multi-class if needed.")

# AUPRC for the training set
if len(np.unique(y_train)) == 2:
    auprc_train = average_precision_score(y_train, y_pred_prob_train[:, 1])
    print(f"AUPRC for Training Set is: {auprc_train}")
else:
    print("AUPRC is typically used for binary classification. Adjust the code for multi-class if needed.")

# Confusion Matrix for the training set
cm_train = confusion_matrix(y_train, y_pred_train)
print(f"Confusion Matrix for Training Set is: \n{cm_train}")


F1 Score for Training Set is: 0.6337464251668257
Accuracy for Training Set is: 0.9218819893456955
Balanced Accuracy for Training Set is: 0.7369427993874346
AUC for Training Set is: 0.9487933938991525
AUPRC for Training Set is: 0.8271620505190398
Confusion Matrix for Training Set is: 
[[21008   116]
 [ 1805  1662]]


In [30]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score, confusion_matrix

# Assumed correction: Ensure y_pred_prob_test is fetched correctly and has two columns
# y_pred_prob_test = model.predict_proba(X_test)  # Correctly fetching test probabilities

# F1 Score
f1 = f1_score(y_test, y_pred_test)
print(f"F1 Score for Testing Set is: {f1}")

# Accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for Testing Set is: {accuracy}")

# Balanced Accuracy
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print(f"Balanced Accuracy for Testing Set is: {balanced_accuracy}")

# AUC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:  # Ensuring binary classification with two probability outputs
    auc = roc_auc_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUC for Testing Set is: {auc}")
else:
    print("Error: Expected binary class probabilities for AUC calculation.")

# AUPRC
if y_pred_prob_test.ndim == 2 and y_pred_prob_test.shape[1] == 2:
    auprc = average_precision_score(y_test, y_pred_prob_test[:, 1])
    print(f"AUPRC for Testing Set is: {auprc}")
else:
    print("Error: Expected binary class probabilities for AUPRC calculation.")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
print(f"Confusion Matrix for Testing Set is: \n{cm}")


F1 Score for Testing Set is: 0.3940242763772176
Accuracy for Testing Set is: 0.8768500948766603
Balanced Accuracy for Testing Set is: 0.6286588798198706
AUC for Testing Set is: 0.8554489696244386
AUPRC for Testing Set is: 0.5165519978746538
Confusion Matrix for Testing Set is: 
[[4410  112]
 [ 537  211]]


In [31]:
est.feature_importances_

# Get the top 10 most important features
indices = np.argsort(est.feature_importances_)[::-1]
top_indices = indices[:100]
print('Feature ranking:')
for i in range(50):
    print('%d. %s (%f)' % (i + 1, col_names[top_indices[i]], est.feature_importances_[top_indices[i]]))

Feature ranking:
1. Diastolic BP_max (0.058471)
2. GCS - Eye Opening_max (0.014327)
3. Respiratory Rate_max (0.009765)
4. Vancomycin_mean (0.009507)
5. Respiratory Rate_mean (0.008515)
6. GCS - Eye Opening_min (0.007363)
7. Vancomycin_min (0.006762)
8. Respiratory Rate_min (0.006646)
9. Absolute Neutrophil Count_mean (0.006516)
10. te_233 (0.005733)
11. Diastolic BP_mean (0.005556)
12. Alkaline Phosphate_mean (0.005449)
13. te_731 (0.005115)
14. Diastolic BP_min (0.004965)
15. Systolic BP_mean (0.004898)
16. te_519 (0.004790)
17. te_107 (0.004631)
18. te_660 (0.004585)
19. Diastolic BP_trend (0.004287)
20. te_280 (0.004283)
21. te_61 (0.004194)
22. Vancomycin_max (0.004176)
23. te_50 (0.004152)
24. Urea Nitrogen_min (0.004097)
25. Creatinine_max (0.003986)
26. te_515 (0.003961)
27. Temperature_sumabsdiff (0.003848)
28. Creatinine_mean (0.003763)
29. te_483 (0.003687)
30. PH_variance (0.003676)
31. Potassium_min (0.003675)
32. te_555 (0.003536)
33. Urea Nitrogen_mean (0.003460)
34. te_4